## 휴먼 인 더 루프 - 프로그램 시스템 사에서 사람의 개입이 필요한 부분

1. 중단점(BreakPoint): AI가 인간의 개입이 필요한 시점을 인식
2. 컨텍스트 제공: 인간이 올바른 결정을 내릴 수 있도록 충분한 정보 제공
3. 피드백 수집: 구조화된 방식으로 인간의 입력 수정
4. 재개(Resume): 인간의 결정에 따라 워크플로 계속 진행

해당 내용은 요즘 ai 에이전트 개발 llM RAG ADK MCP LangChain A2A (저자: 박승규) 에서 첨부하였습니다.

In [1]:
# 필요한 라이브러리
from typing import Literal
from langgraph.graph import StateGraph, START,END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

In [2]:
#Pydantic을 사용한 state 정의 
class AgentState(BaseModel):
    user_message: str = Field(default="", description="사용자 입력 작업")
    task_details: str = Field(default="", description="직업 상세 정보")
    response: str = Field(default="", description="응답 결과")

In [3]:
#노드 함수 정의
def get_llm_response_node(state: AgentState, llm):
    """LLM과 상호작용하여 응답을 생성하거나, 추가 정보를 요청하는 노드"""
    details = state.task_details
    
    if details:
        print(f"\n상세 정보를 바탕으로 작업 실행: {details}")
        prompt = f"다음 요청에 따라 보고서를 작성해주세요: {details}"
    else:
        task = state.user_message
        print(f"\n작업 실행: {task} 작업을 수행합니다...")
        # ② LLM에게 상세 정보를 묻는 질문을 하도록 유도하고, 반드시 '?'로 끝내도록 지시
        prompt = f"'{task}' 작업을 수행하려고 합니다. 어떤 종류의 보고서가 필요한지, 구체적인 주제는 무엇인지 질문해주세요. 추가 정보가 필요하면, 반드시 응답의 마지막을 물음표('?')로 끝내주세요."
    
    response = llm.invoke(prompt).content
    
    print("---- LLM 응답 ----")
    print(response)
    print("------------------")
    
    return {"response": response, "task_details": ""}

In [4]:
# ③ 사람의 입력을 받는 노드
def get_task_details_node(state:AgentState) -> AgentState:
    """LLM의 질문에 대한 사용자 답변을 입력받는 노드"""
    print("\nLLM의 질문에 답변해주세요.")
    user_input = input("답변: ")
    return {"task_details" : user_input}

In [5]:
#조건부 분기 노드
def check_llm_response(state: AgentState) -> Literal["get_details", "end"]:
    """LLM의 응답이 질문인지 확인하여 다음 단계를 결정합니다."""
    print("LLM 응답 분석 중.....")
    if state.response.strip().endswith('?'):
        print("LLM이 추가 정보를 요청했습니다. 사용자 입력을 받습니다.")
        return "get_details"
    print("최종 보고서가 생성되었습니다. 워크플로우를 종료합니다.")
    return "end"

In [6]:
def create_graph():
    """human-in-the-loop 워크플로우 그래프를 생성합니다."""
    # 그래프 전체에서 사용할 LLM 모델을 초기화 합니다.
    llm = init_chat_model("gpt-5-mini", model_provider="openai")
    
    def get_llm_response_with_llm(state):
        return get_llm_response_node(state, llm)
    
    workflow = StateGraph(AgentState)
    workflow.add_node("get_llm_response", get_llm_response_with_llm)
    workflow.add_node("get_details", get_task_details_node)
    
    workflow.add_edge(START, "get_llm_response")
    workflow.add_conditional_edges(
        "get_llm_response",
        check_llm_response,
        {
            "get_details":"get_details",
            "end" : END
        }
    )
    workflow.add_edge("get_details", "get_llm_response")
    return workflow.compile()

In [7]:
if __name__ == "__main__":
     print("=== LangGraph Human-in-the-loop 간소화 예제 ===\n")
     app = create_graph()
     final_state = app.invoke(AgentState(user_message="블로그 글 작성"))
     print("\n--- 워크플로우 종료 ---")
     print("최종 응답:")
     print(final_state["response"])

=== LangGraph Human-in-the-loop 간소화 예제 ===


작업 실행: 블로그 글 작성 작업을 수행합니다...
---- LLM 응답 ----
블로그 글 작성을 정확히 도와드리려면 아래 항목들을 알려주세요:

- 어떤 종류의 글(정보성/분석/가이드/리뷰/사례연구/SEO용/랜딩페이지 등)을 원하시나요?  
- 구체적인 주제나 핵심 키워드는 무엇인가요?  
- 목표 독자(연령대, 관심사, 전문성 수준 등)는 어떻게 되나요?  
- 글의 목표는 무엇인가요?(트래픽 증가, 리드 생성, 브랜드 인지도, 제품 홍보 등)  
- 원하는 글 길이(대략 단어 수 또는 분량)는 얼마인가요?  
- 톤과 스타일 선호(친근한/전문적인/중립적/카주얼 등)는 무엇인가요?  
- 반드시 포함해야 할 핵심 포인트나 메시지가 있나요?  
- 목차나 섹션 구성에 대한 선호가 있나요(예: 서론/문제제기/해결책/사례/결론 등)?  
- SEO 요구사항(주요 키워드, 보조 키워드, 메타디스크립션, 제목 태그 등)이 있나요?  
- 이미지, 차트, 표 등 시각 자료가 필요한가요? 직접 제공하실 건가요, 제가 추천할까요?  
- 참고할 자료나 인용해야 할 출처(URL, 논문, 내부 문서 등)이 있나요?  
- 내부 링크 또는 외부 링크 연결 정책(특정 페이지로 연결 등)이 있나요?  
- 인용 형식(APA, MLA 등)이나 각주 필요 여부는요?  
- 표절 검사나 원본성 기준(예: 유사도 몇 % 이하)이 있나요?  
- 납기일(완료 희망일)과 수정 가능 횟수 제약이 있나요?  
- 참고할 예시 글이나 선호하는 블로그(스타일 예시) URL이 있다면 알려주실 수 있나요?  
- 다국어 번역이나 국제화(영어 버전 등)가 필요하신가요?  
- 해당 글의 공개 범위는 내부용인가요, 외부 공개(게시)용인가요?  
- 추가로 특별히 원하시는 사항이나 제약 조건이 있나요?

답변을 주시면 그에 맞춰 초안 구조와 예상 소요시간을 제안드리겠습니다?
------------------
LLM 응답 분석